In [2]:
import sqlite3
from tkinter import*
from tkinter import ttk, messagebox
from PIL import ImageTk 

In [3]:
class Database:
    
    def __init__(self, db): # pass self to avoid TypeError: '...takes 0 positional arguments but 1 was given.'
        """
        Initialise the object, when calling the class. Connecting to the database.
        """
        self.conn=sqlite3.connect('books.db')
        self.cur = self.conn.cursor() # assigning cursor object to the class attribute.
        self.cur.execute("CREATE TABLE IF NOT EXISTS books (id INTEGER PRIMARY KEY, title text, author text, year integer, Category text)")
        self.conn.commit()
        # removed the close method.

    def insert(self,title, author, year, category):
        self.cur.execute("INSERT INTO books VALUES (NULL,?,?,?,?)", (title, author, year, category))
        self.conn.commit()
        #conn.close() # removing the close() 

    def view(self):
        self.cur.execute("SELECT * FROM books")
        rows=self.cur.fetchall()
        return rows

    def search(self,title="", author="", year="", category=""):
        self.cur.execute("SELECT * FROM books WHERE title=? OR author=? OR year=? OR Category=?", (title, author, year, category))
        rows=self.cur.fetchall()
        return rows

    def delete(self,id):
        # delete something from book.db where id is equal to something.
        self.cur.execute("DELETE FROM books WHERE id=?", (id,))
        self.conn.commit()
        #conn.close()

    def __del__(self):
        """
        Closing the database connection.
        """
        self.conn.close()

In [4]:
"""
A Program that stores book's information.
Front-End:
    Text (Entry Widgets) Fields: Title, Author, Year, CATEGORY
    User can view all records, search entry and exit (program).
Back-End:
    Entry Widget
    Scroll Bar
    Command Buttons
    Tkinter -> Grid Method
"""
#from backend import Database # importing the class

database = Database("books.db")

class Window(object):

    def __init__(self,window):
        self.window = window
        self.window.geometry('1100x500+150+100')
        self.window.wm_title("Book Store")
        
        self.bg=ImageTk.PhotoImage(file='bg1.jpg')
        bg_lbl=Label(self.window,image=self.bg,bd=2,relief=RAISED)   #relief=RAISED==> we give 3D effect to our main window 
        bg_lbl.place(x=0,y=0,relwidth=1,relheight=1)

        l1=Label(window,text="    Title    ",font=('Times New Roman',25,'bold'),fg='black',bg='yellow')
        l1.grid(row=0,column=0,padx=100,pady=30,sticky=W)

        l2=Label(window,text="  Author  ",font=('Times New Roman',25,'bold'),fg='black',bg='yellow')
        l2.grid(row=0,column=2,padx=10,pady=30,sticky=W)

        l3=Label(window,text="    Year    ",font=('Times New Roman',25,'bold'),fg='black',bg='yellow')
        l3.grid(row=1,column=0,padx=100,pady=30,sticky=W)

        l4=Label(window,text="Category",font=('Times New Roman',25,'bold'),fg='black',bg='yellow')
        l4.grid(row=1,column=2)
        
        l5=Label(window,text="id | TITLE | AUTHOR | YEAR | CATEGORY ",font=('Times New Roman',14,'bold'),fg='black',bg='yellow')
        l5.place(x=40,y=200)

        self.title_text=StringVar()
        self.e1=Entry(window,textvariable=self.title_text,font=('Times New Roman',25,'bold'),width=15)
        self.e1.grid(row=0,column=1)

        self.author_text=StringVar()
        self.e2=Entry(window,textvariable=self.author_text,font=('Times New Roman',25,'bold'),width=15)
        self.e2.grid(row=0,column=3)

        self.year_text=StringVar()
        self.e3=Entry(window,textvariable=self.year_text,font=('Times New Roman',25,'bold'),width=15)
        self.e3.grid(row=1,column=1)

        self.category_text=StringVar()
        self.e4=Entry(window,textvariable=self.category_text,font=('Times New Roman',25,'bold'),width=15)
        self.e4.grid(row=1,column=3)

        self.list1=Listbox(window, height=6,width=60,font=('Times New Roman',13,'italic'))
        self.list1.grid(row=3,column=0,padx=20,pady=30,rowspan=7,columnspan=2)

        self.sb1=Scrollbar(window)
        self.sb1.grid(row=3,column=2,rowspan=6)

        self.list1.configure(yscrollcommand=self.sb1.set)
        self.sb1.configure(command=self.list1.yview)

        # bind method for the delete_command
        self.list1.bind("<<ListboxSelect>>", self.get_selected_row)

        b1=Button(window, text='View All', command=self.view_command, width=15,bg='purple',fg='white',font=('bold',15),cursor='hand2')
        b1.grid(row=3, column=3)

        b2=Button(window, text='Search Entry', command=self.search_command, width=15,bg='purple',fg='white',font=('bold',15),cursor='hand2')
        b2.grid(row=4, column=3)

        b6=Button(window, text='Close', command=window.destroy, width=15,bg='purple',fg='white',font=('bold',15),cursor='hand2')
        b6.grid(row=8, column=3)

    def get_selected_row(self,event):
        
        #global selected_tuple
        # identifies the index of the user selected item in the list1. 
        index=self.list1.curselection()[0]
        self.selected_tuple=self.list1.get(index)
    
        # Shows in the entry fields the user selected row item.
        self.e1.delete(0,END)
        self.e1.insert(END,self.selected_tuple[1]) # title has the index of 1
        self.e2.delete(0,END)
        self.e2.insert(END,self.selected_tuple[2]) # title has the index of 1
        self.e3.delete(0,END)
        self.e3.insert(END,self.selected_tuple[3]) # title has the index of 1
        self.e4.delete(0,END)
        self.e4.insert(END,self.selected_tuple[4]) # title has the index of 1    
        #print(selected_tuple)

    def view_command(self):
        # ensures that everything is deleted from 0 to the end (last row). Pressing 'view all' does not repeatedly shows the same results.
        self.list1.delete(0,END)
        for row in database.view():
            # the new rows will be added at the end of the exisiting rows. Ensures that every new rows are added at the end.
            self.list1.insert(END,row)

    def search_command(self):
        self.list1.delete(0, END)
        # uses get for a search method.
        for row in database.search(self.title_text.get(), self.author_text.get(), self.year_text.get(), self.category_text.get()):
            self.list1.insert(END, row)

    def delete_command(self):
        # once the user selects the row, we need to identify the ID and get it deleted.
        database.delete(self.selected_tuple[0])

In [6]:
if __name__=='__main__':
    window = Tk()
    Window(window)
    window.mainloop()